# Lesson: The Core of Agency - Function Calling

Welcome to this lesson on **Function Calling**, one of the most powerful features of the Gemini API. This is how you give your AI "hands" to interact with the world, transforming it from a simple chatbot into a capable agent.

### The Concept
Function calling allows you to define a set of your own Python functions (we'll call them "tools") and make them available to the Gemini model. The model can then intelligently decide when to use these tools based on the user's prompt. It doesn't execute the function itself; instead, it asks *your code* to run the function with the correct arguments.

In this notebook, we will:
1.  **Build a Smart Home Controller:** A simple agent that can turn lights on or off and play music.
2.  **Handle Function Calls:** Learn the complete loop of receiving a function call request, executing your code, and sending the result back to the model.
3.  **Implement a Multi-Tool Agent:** Create an agent that has multiple tools and can choose the correct one for the user's request.
4. **Compositional Function Calls:** An advanced agent that can chain tools together, using the output of one as the input for another.

In [5]:
#@title 1. Setup
# Install the Google AI Python SDK
!pip install -q -U google-genai

In [6]:
#@title 2. Configure your API Key
# Use the "Secrets" tab in Colab (click the key icon on the left) to store your
# API key with the name "GOOGLE_API_KEY".

from google import genai
from google.colab import userdata

try:
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    client = genai.Client(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
    print('Secret not found. Please add your GOOGLE_API_KEY to the Colab Secrets Manager.')

## Part 1: Building a Smart Home Controller

Let's start by defining some simple Python functions that will act as our "tools" for controlling a smart home.

In [8]:
#@title 1. Define the Tools (Python Functions)

# This dictionary will simulate the state of our smart home devices
smart_home_state = {
    "living_room_light": False,
    "speaker_playing": None
}

def set_light_state(room: str, state: bool):
    """Turns a light on or off in a specified room."""
    print(f"--- TOOL: Turning {room} light {'ON' if state else 'OFF'} ---")
    smart_home_state[f"{room}_light"] = state
    return {"status": "success", "room": room, "state": state}

def play_music(artist: str, song: str):
    """Plays a song by a specified artist."""
    print(f"--- TOOL: Playing '{song}' by {artist} ---")
    smart_home_state["speaker_playing"] = f"{song} by {artist}"
    return {"status": "success", "now_playing": f"{song} by {artist}"}

smart_home_tools = [set_light_state, play_music]

print("Smart Home tools defined successfully.")

Smart Home tools defined successfully.


In [9]:
#@title 2. Make the Tools Available to the Model

# Function calls naturally fit into multi-turn conversations. The Python SDK's ChatSession (client.chats.create(...)) is ideal for this, as it automatically handles conversation history.
# Furthermore, ChatSession simplifies function calling execution via its automatic_function_calling feature (enabled by default)

prompt = "Can you turn on the light in the living room and then play 'Bohemian Rhapsody' by Queen?"

chat = client.chats.create(
    model="gemini-2.5-flash",
    config={
        "tools": smart_home_tools
    }
)

response = chat.send_message(prompt)

print(response.text)

--- TOOL: Turning living room light ON ---
--- TOOL: Playing 'Bohemian Rhapsody' by Queen ---
I've turned on the light in the living room and started playing 'Bohemian Rhapsody' by Queen.


In [10]:
#@title 3. Examining Function Calls and Execution History
# To understand what happened in the background, you can examine the chat history.

# The Chat.history property stores a chronological record of the conversation between the user and the Gemini model. You can get the history using Chat.get_history(). Each turn in the conversation is represented by a genai.types.Content object, which contains the following information:

# Role: Identifies whether the content originated from the "user" or the "model".

# Parts: A list of genai.types.Part objects that represent individual components of the message. With a text-only model, these parts can be:

# Text: Plain text messages.
# Function Call (genai.types.FunctionCall): A request from the model to execute a specific function with provided arguments.
# Function Response (genai.types.FunctionResponse): The result returned by the user after executing the requested function.

from IPython.display import Markdown, display

def print_history(chat):
  for content in chat.get_history():
      display(Markdown("###" + content.role + ":"))
      for part in content.parts:
          if part.text:
              display(Markdown(part.text))
          if part.function_call:
              print("Function call: {", part.function_call, "}")
          if part.function_response:
              print("Function response: {", part.function_response, "}")
      print("-" * 80)

print_history(chat)

###user:

Can you turn on the light in the living room and then play 'Bohemian Rhapsody' by Queen?

--------------------------------------------------------------------------------


###model:

Function call: { id=None args={'room': 'living room', 'state': True} name='set_light_state' }
--------------------------------------------------------------------------------


...

###user:

Function response: { will_continue=None scheduling=None parts=None id=None name='play_music' response={'result': {'status': 'success', 'now_playing': 'Bohemian Rhapsody by Queen'}} }
--------------------------------------------------------------------------------


###model:

I've turned on the light in the living room and started playing 'Bohemian Rhapsody' by Queen.

--------------------------------------------------------------------------------


## Part 2: Compositional Reasoning - Chaining Function Calls

Now for the most advanced pattern: compositional reasoning. This is when the model needs to chain function calls together, using the output of one function as the input for another to solve a multi-step problem.

We'll build an **"E-commerce Assistant"** that can first find a product's ID and then use that ID to look up its reviews.

In [12]:
#@title 1. Define the Tools for the E-commerce Assistant
# Simulate a product database
product_db = {
    "Pixel 8 Pro": { "product_id": "P8PRO-2024" },
    "Pixel Tablet": { "product_id": "PTAB-2024" }
}

# Simulate a reviews database
reviews_db = {
    "P8PRO-2024": ["Amazing camera!", "The battery life is excellent.", "A bit pricey but worth it."],
    "PTAB-2024": ["Great for media consumption.", "The speaker dock is a game-changer."]
}

def search_product(product_name: str):
    """Finds the product_id for a given product name."""
    print(f"--- TOOL: Searching for product: '{product_name}' ---")
    product = product_db.get(product_name)
    if product:
        return {"product_id": product["product_id"]}
    else:
        return {"error": "Product not found."}

def get_product_reviews(product_id: str):
    """Gets the customer reviews for a given product_id."""
    print(f"--- TOOL: Fetching reviews for product_id: '{product_id}' ---")
    reviews = reviews_db.get(product_id)
    if reviews:
        return {"reviews": reviews}
    else:
        return {"error": "Reviews not found for this product_id."}

product_tools = [search_product, get_product_reviews]

print("E-commerce tools defined successfully.")

E-commerce tools defined successfully.


In [13]:
#@title 2. Make the Tools Available to the Model

prompt = "First find the product ID for the 'Pixel 8 Pro', and then use that ID to tell me what its customer reviews are."

chat = client.chats.create(
    model="gemini-2.5-flash",
    config={
        "tools": product_tools
    }
)

response = chat.send_message(prompt)

print(response.text)


--- TOOL: Searching for product: 'Pixel 8 Pro' ---
--- TOOL: Fetching reviews for product_id: 'P8PRO-2024' ---


The customer reviews for the Pixel 8 Pro are: "Amazing camera!", "The battery life is excellent.", and "A bit pricey but worth it."


In [15]:
#@title 3. Examining Function Calls and Execution History
print_history(chat)

###user:

First find the product ID for the 'Pixel 8 Pro', and then use that ID to tell me what its customer reviews are.

--------------------------------------------------------------------------------


###model:

Function call: { id=None args={'product_name': 'Pixel 8 Pro'} name='search_product' }
--------------------------------------------------------------------------------


...

###user:

Function response: { will_continue=None scheduling=None parts=None id=None name='get_product_reviews' response={'result': {'reviews': ['Amazing camera!', 'The battery life is excellent.', 'A bit pricey but worth it.']}} }
--------------------------------------------------------------------------------


###model:

The customer reviews for the Pixel 8 Pro are: "Amazing camera!", "The battery life is excellent.", and "A bit pricey but worth it."

--------------------------------------------------------------------------------
